In [17]:
import numpy as np
import navtools as nt
from scipy.linalg import inv, norm, eig, svd, det

LIGHT_SPEED = 299792458.0  #! [m/s]
FREQUENCY = 1575.42e6  #! [Hz]
CHIP_FREQ = 1.023e6  #! [chips/s]
WAVELENGTH = LIGHT_SPEED / FREQUENCY  #! [m]
CHIP_WIDTH = LIGHT_SPEED / CHIP_FREQ  #! [m/chip]

R2D = 180 / np.pi
LLA_R2D = np.array([R2D, R2D, 1], dtype=float)

In [18]:
##### *SETUP* #####
CN0 = 50
TRUE_AZ = np.array([77.62,  163.93,   55.5 ,  -73.75,  -20.07,  -94.36, -114.59, 49.77, -142.6])
TRUE_EL = np.array([53.82, 40.59,  6.16, 19.19, 59.38, 55.58, 26.75, 33.31, 71.03])
L = TRUE_AZ.size

ant_body = (
    np.array(
        [
            [0, 0, 0],
            [WAVELENGTH, 0, 0],
            [0, -WAVELENGTH, 0],
            [WAVELENGTH, -WAVELENGTH, 0],
        ],
        dtype=float,
    )
    / 2.0
)
ant_att_rpy = np.zeros(3)
ant_att_rpy = np.array([0, 0, -30], dtype=float)
C_b_n = nt.euler2dcm(ant_att_rpy / R2D, "enu")
ant_xyz = np.zeros(ant_body.shape)
for i in range(ant_xyz.shape[0]):
    ant_xyz[i, :] = C_b_n @ ant_body[i, :]


In [24]:
##### *CORRELATORS* #####
true_u = np.array([np.sin(TRUE_AZ / R2D) * np.cos(TRUE_EL / R2D), np.cos(TRUE_AZ / R2D) * np.cos(TRUE_EL / R2D),  np.sin(TRUE_EL / R2D)])
spatial_phase = np.array([ant_xyz @ true_u[:, i] for i in range(true_u.shape[1])]).T

X = np.zeros(spatial_phase.shape, dtype=complex)
phase_err = 0.0 * np.random.randn(L) - spatial_phase / WAVELENGTH
chip_err = 0.0 * np.random.randn(L) - spatial_phase / CHIP_WIDTH
freq_err = 0.0 * np.random.randn(L) + spatial_phase / WAVELENGTH * 0.02
print(phase_err.T)

A = np.sqrt(2.0 * 10 ** (CN0 / 10) * 0.02)
R = 1.0 - np.abs(chip_err)
R[R <= 0.0] = 0.0
F = np.sinc(np.pi * freq_err * 0.02)
P = np.exp(1j * 2.0 * np.pi * phase_err)
tmp = A * R * F * P
X.real = tmp.real #+ np.random.randn(X.shape[0], X.shape[1])
X.imag = tmp.imag #+ np.random.randn(X.shape[0], X.shape[1])

[[-0.          0.0893463   0.28131447  0.37066077]
 [-0.          0.3685259  -0.09140574  0.27712016]
 [ 0.         -0.03900304  0.49558063  0.45657758]
 [ 0.         -0.34111245 -0.32654414 -0.66765659]
 [ 0.         -0.25085574  0.0439167  -0.20693904]
 [ 0.         -0.12229722 -0.25479735 -0.37709457]
 [-0.         -0.04209595 -0.4445006  -0.48659655]
 [ 0.         -0.0742112   0.41121301  0.33700181]
 [ 0.          0.06246202 -0.15005537 -0.08759335]]


In [20]:
##### *MUSIC* #####
AZ_EST = np.zeros(TRUE_AZ.shape)
EL_EST = np.zeros(TRUE_EL.shape)

for l in range(L):
    M = X.shape[0]
    S = np.outer(X[:, l], X[:, l].conj()) / M  #! MxM covariance of X, Eq. 2
    e, v = eig(S)
    idx = np.abs(e) < (e.max() / 10)
    N = int(np.sum(idx))
    D = M - N  #! number of incident signals, Eq. 5
    U = v[:, idx]  #! null space of S

    k = 0
    res = 10.0
    az_mean = 0.0
    el_mean = 0.0
    az = np.arange(-180.0, 190.0, res)
    el = np.arange(-90.0, 100.0, res)
    P_music = np.zeros((az.size, el.size), dtype=float)
    while res > 0.01:
        if k > 0:
            span = res / 2
            new_res = res / 10
            az = np.arange(az_mean - span, az_mean + span + new_res, new_res)
            el = np.arange(el_mean - span, el_mean + span + new_res, new_res)
            res = new_res
            P_music = np.zeros((az.size, el.size), dtype=float)
        for i in range(az.size):
            for j in range(el.size):
                u = np.array(
                    [
                        np.sin(az[i] / R2D) * np.cos(el[j] / R2D), 
                        np.cos(az[i] / R2D) * np.cos(el[j] / R2D), 
                        np.sin(el[j] / R2D)
                    ]
                )
                a = np.exp(-1j * 2.0 * np.pi * (ant_body @ u) / WAVELENGTH)
                P_music[i, j] = 1.0 / np.abs((a.conj().T @ U @ U.conj().T @ a))  #! 1 / euclidean distance, Eq. 6
        az_idx, el_idx = np.unravel_index(P_music.argmax(), P_music.shape)

        az_mean = az[az_idx]
        el_mean = el[el_idx]
        k += 1

    AZ_EST[l] = az_mean
    EL_EST[l] = el_mean

print(f"true_az = {TRUE_AZ}")
print(f"est_az  = {np.round(AZ_EST,2)} \n")
print(f"true_el = {TRUE_EL}")
print(f"est_el  = {np.round(EL_EST,2)} \n")

true_az = [  77.62  163.93   55.5   -73.75  -20.07  -94.36 -114.59   49.77 -142.6 ]
est_az  = [ -17.62 -107.05    4.41  133.75   80.07  154.36  174.59   10.23 -157.4 ] 

true_el = [53.82 40.59  6.16 19.19 59.38 55.58 26.75 33.31 71.03]
est_el  = [-53.82 -35.55  -7.45 -19.19 -59.38 -55.58 -26.75 -33.31 -71.03] 



In [21]:
##### *INITIAL ATTITUDE ESTIMATE* #####
R_inv = inv(2.0 / R2D * np.eye(L))
# R_inv = np.eye(L)
D_loc = np.array(
    [
        np.sin(AZ_EST / R2D) * np.cos(EL_EST / R2D),
        np.cos(AZ_EST / R2D) * np.cos(EL_EST / R2D),
        np.sin(EL_EST / R2D),
    ],
)
D_enu = np.array(
    [
        np.sin(TRUE_AZ / R2D) * np.cos(TRUE_EL / R2D),
        np.cos(TRUE_AZ / R2D) * np.cos(TRUE_EL / R2D),
        np.sin(TRUE_EL / R2D),
    ],
)
M_hat = D_loc @ R_inv @ D_enu.T @ inv(D_enu @ R_inv @ D_enu.T)
rpy_hat = nt.dcm2euler(M_hat, "enu") * R2D
# rpy_hat = np.array(
#         [
#             -np.arctan2(M_hat[2, 1], -M_hat[2, 2]), 
#             np.arcsin(M_hat[2, 0]), 
#             # np.arctan2(M_hat[2, 0], norm(M_hat[[0, 0], [1, 0]])),
#             np.arctan2(M_hat[0, 0], M_hat[1, 0])
#         ]
#       ) * R2D


B = D_loc @ R_inv @ D_enu.T
U, _, V = svd(B)
d = det(U) * det(V.T)
M_svd = U @ np.diag([1.0,1.0,d]) @ V
rpy_svd = nt.dcm2euler(M_svd, "enu") * R2D
# rpy_svd = np.array(
#         [
#             -np.arctan2(M_svd[2, 1], -M_svd[2, 2]), 
#             np.arcsin(M_svd[2, 0]), 
#             # np.arctan2(M_svd[2, 0], norm(M_svd[[0, 0], [1, 0]])),
#             np.arctan2(M_svd[0, 0], M_svd[1, 0])
#         ]
#       ) * R2D

print(f"True Attitude = {np.round(ant_att_rpy,2)}")
print(f"Est. Attitude = {np.round(rpy_hat,2)}")
print(f"SVD  Attitude = {np.round(rpy_svd,2)} \n")

True Attitude = [  0.   0. -30.]
Est. Attitude = [  2.5    0.6  -30.81]
SVD  Attitude = [  1.08  -0.06 -30.5 ] 



In [22]:
##### *RECURSIVE ATTITUDE REFINEMENT* #####